# 语义特征

In [1]:
import numpy as np
import pandas as pd

# 加载 .npy 文件
npy_file = "E:/comorbidity/feature_list/emb-chinese-mentalbert.npy"  # 替换为你的 .npy 文件路径
data = np.load(npy_file)

# 将 numpy 数组转换为 pandas DataFrame
df = pd.DataFrame(data)

# 保存为 .csv 文件
csv_file = 'E:/comorbidity/feature_normalization/emb-cn-mentalbert.csv'  # 替换为你想要保存的 .csv 文件路径
df.to_csv(csv_file, index=False, encoding='utf-8-sig')  # index=False 让 DataFrame 保存时不包含行索引

# 主题特征

In [6]:
from gensim.models import LdaModel
import csv
import numpy as np

# 加载模型
ldaseq_loaded = LdaModel.load("E:/comorbidity/feature_list/DTM-5个.gensim")

# 打印模型的一些信息，检查是否加载成功
print(ldaseq_loaded.print_topics())

[[('症状', 0.022700551007552694), ('患者', 0.022598571136687413), ('焦虑', 0.02176656195107239), ('抑郁', 0.0162725228168398), ('中医', 0.01593125901118447), ('医生', 0.015524677480199041), ('失眠', 0.015380379507398009), ('治疗', 0.014583489341469871), ('发作', 0.011506294309164986), ('心理', 0.010576132445706187), ('情绪', 0.009065834026221145), ('紊乱', 0.008386928101896798), ('躯体', 0.007530410950713271), ('植物神经', 0.007337360892952207), ('医院', 0.006897937566009267), ('药物', 0.0066866380504074585), ('惊恐', 0.006593622329989141), ('障碍', 0.006207823519127699), ('导致', 0.005628011974797442), ('疾病', 0.005396643705374734)], [('希望', 0.021193113939525027), ('啊啊啊', 0.018715660688548033), ('微博', 0.018631603962450945), ('视频', 0.01715612451993435), ('患者', 0.014132016600130144), ('世界', 0.01015462202009507), ('开心', 0.009352838488973565), ('健康', 0.009182531261986298), ('快乐', 0.009118114081749342), ('孩子', 0.008349244634337219), ('喜欢', 0.007233896602095201), ('生活', 0.006629474379980719), ('好好', 0.0063702173641513446), ('一年', 

In [7]:

# 获取主题数量
num_topics = ldaseq_loaded.num_topics
total_docs = 95163  # 总文档数

# 生成动态表头
header = ["文档ID"] + [f"主题{i}" for i in range(num_topics)] + ["主主题"]

with open("E:/comorbidity/feature_normalization/doc_topic_5.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(header)  # 写入新表头
    
    for doc_id in range(total_docs):
        # 获取主题概率分布
        topic_dist = ldaseq_loaded.doc_topics(doc_id)
        
        # 转换为概率列表
        prob_list = [round(prob, 4) for prob in topic_dist]  # 保留4位小数
        
        # 找到主主题
        main_topic = topic_dist.argmax()
        
        # 组合行数据
        row = [doc_id] + prob_list + [main_topic]
        writer.writerow(row)

print(f"处理完成！结果已保存到 doc_topic_5.csv")

处理完成！结果已保存到 doc_topic_5.csv


# 情绪特征（0-1标准化）

In [13]:
#pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [6]:
import chardet
with open("E:/comorbidity/feature_list/cnsenti情绪结果.csv", 'rb') as f:
    rawdata = f.read(10000)  # 读取前10000字节用于检测
    encoding = chardet.detect(rawdata)['encoding']
print(f"检测到编码格式: {encoding}")

检测到编码格式: ISO-8859-1


In [11]:
import pandas as pd

# 指定文件路径
file_path = "E:/comorbidity/feature_list/cnsenti情绪结果.csv"
output_path = "E:/comorbidity/feature_normalization/emotion_results.csv"

# 定义情感列
emotion_columns = ['好', '乐', '哀', '怒', '惧', '恶', '惊']

# 1. 读取文件（明确使用GB2312编码）
df = pd.read_csv(file_path, encoding='GB2312')

# 2. 检查列名是否匹配
missing_cols = [col for col in emotion_columns if col not in df.columns]
if missing_cols:
    raise ValueError(f"CSV文件中缺少以下列: {missing_cols}")

# 3. 归一化处理
df_normalized = df.copy()
row_sums = df_normalized[emotion_columns].sum(axis=1)
safe_row_sums = row_sums.replace(0, 1)  # 避免除零错误
df_normalized[emotion_columns] = df_normalized[emotion_columns].div(safe_row_sums, axis=0).round(4)

# 4. 保存结果（使用通用编码）
df_normalized.to_csv(output_path, index=False, encoding='utf-8-sig')

# 输出结果
print("="*50)
print(f"成功处理 {len(df)} 条数据！")
print(f"结果文件已保存至: {output_path}")


成功处理 95163 条数据！
结果文件已保存至: E:/comorbidity/feature_normalization/emotion_results.csv


# 情感极性+情感强度

In [8]:
import pandas as pd

def process_emotion_data(input_path, output_path):
    """
    处理情感分析数据并输出CSV文件
    参数：
    input_path: 输入Excel文件路径
    output_path: 输出CSV文件路径
    """
    try:
        # 1. 读取Excel文件
        df = pd.read_excel(input_path)
        
        # 2. 验证数据列是否存在
        required_columns = ['最终的情感分析结果', '情感强度']
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise ValueError(f"文件缺少必要列：{', '.join(missing_cols)}")

        # 3. 数据预处理
        # 确保情感强度为数值类型
        df['情感强度'] = pd.to_numeric(df['情感强度'], errors='coerce')
        
        # 4. One-Hot编码
        onehot = pd.get_dummies(
            df['最终的情感分析结果'],
            prefix='',
            prefix_sep='',
            columns=['负向', '中性', '正向']  # 显式指定所有可能类别
        ).reindex(columns=['负向', '中性', '正向'], fill_value=0)
        
        # 5. 情感强度归一化
        min_val = df['情感强度'].min()
        max_val = df['情感强度'].max()
        range_val = max_val - min_val
        
        if range_val == 0:
            # 处理全零或恒定值情况
            normalized = 0.0
        else:
            normalized = (df['情感强度'] - min_val) / range_val
        
        # 6. 创建结果DataFrame
        result = pd.concat([
            onehot,
            normalized.rename('情感强度_归一化')
        ], axis=1)
        
        # 7. 保存结果
        result.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        print("="*40)
        print(f"成功处理 {len(df)} 条数据")
        print("结果文件列结构：")
        print(result.head(3).to_markdown(index=False))
        print(f"文件已保存至：{output_path}")
        
    except Exception as e:
        print("="*40)
        print("处理失败，错误信息：")
        print(str(e))
        print("="*40)
        print("常见问题排查：")
        print("1. 确认Excel文件路径是否正确")
        print("2. 检查Excel文件是否被其他程序打开")
        print("3. 验证数据列是否包含中文标点")

# 使用示例
input_excel = "E:/comorbidity/feature_list/修正BOSON情感分析结果+情感强度.xlsx"  # 修改为实际路径
output_csv = "E:/comorbidity/feature_normalization/sentiment_results.csv"

process_emotion_data(input_excel, output_csv)

成功处理 95163 条数据
结果文件列结构：
| 负向   | 中性   | 正向   |   情感强度_归一化 |
|:-------|:-------|:-------|------------------:|
| True   | False  | False  |          0.2      |
| True   | False  | False  |          0        |
| True   | False  | False  |          0.266667 |
文件已保存至：E:/comorbidity/feature_normalization/sentiment_results.csv


# 社交行为特征——帖子层面

In [4]:
import pandas as pd

def process_social_data(input_path, output_path):
    """
    处理社交媒体数据并输出CSV文件
    参数：
    input_path: 输入Excel文件路径
    output_path: 输出CSV文件路径
    """
    try:
        # ========== 数据读取 ==========
        df = pd.read_excel(input_path, engine='openpyxl')
        
        # ========== 数据验证 ==========
        required_columns = ['数据来源', '转发', '评论', '点赞']
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            raise ValueError(f"文件缺少必要列：{', '.join(missing_cols)}")

        # ========== 数据处理 ==========
        # 1. 数据来源One-Hot编码
        source_encoded = pd.get_dummies(
            df['数据来源'], 
            prefix='来源'
        )
        
        # 2. 数值列归一化（列归一化）
        numeric_cols = ['转发', '评论', '点赞']
        df_normalized = df[numeric_cols].copy()
        
        for col in numeric_cols:
            min_val = df_normalized[col].min()
            max_val = df_normalized[col].max()
            
            # 处理特殊值
            if max_val == min_val:
                df_normalized[col] = 0.0
            else:
                df_normalized[col] = (df_normalized[col] - min_val) / (max_val - min_val)
            
            df_normalized[col] = df_normalized[col].round(4)

        # ========== 结果合并 ==========
        result = pd.concat([source_encoded, df_normalized], axis=1)
        
        # ========== 结果保存 ==========
        result.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        # ========== 输出验证 ==========
        print("="*50)
        print(f"成功处理 {len(df)} 条数据")
        print("各列范围：")
        print(f"转发：{df['转发'].min()}~{df['转发'].max()} → 归一化范围：{df_normalized['转发'].min():.2f}~{df_normalized['转发'].max():.2f}")
        print(f"评论：{df['评论'].min()}~{df['评论'].max()} → 归一化范围：{df_normalized['评论'].min():.2f}~{df_normalized['评论'].max():.2f}")
        print(f"点赞：{df['点赞'].min()}~{df['点赞'].max()} → 归一化范围：{df_normalized['点赞'].min():.2f}~{df_normalized['点赞'].max():.2f}")
        print(f"文件已保存至：{output_path}")
        
    except Exception as e:
        print("="*50)
        print("处理失败，错误信息：")
        print(str(e))
        print("="*50)
        print("排查建议：")
        print("1. 检查Excel中是否包含非数字字符")
        print("2. 确认数值列数据为整数型")

# 正确调用方式（参数名与定义一致）
process_social_data(
    input_path="E:/comorbidity/data/final_data.xlsx",
    output_path="E:/comorbidity/feature_normalization/post.csv"
)

成功处理 95163 条数据
各列范围：
转发：0~1000 → 归一化范围：0.00~1.00
评论：0~307 → 归一化范围：0.00~1.00
点赞：0~5016 → 归一化范围：0.00~1.00
文件已保存至：E:/comorbidity/feature_normalization/post.csv


# 社交行为特征——用户层面

In [2]:
import pandas as pd

def process_excel_data(input_path, output_path):
    """
    处理Excel社交媒体数据
    :param input_path: 输入Excel文件路径
    :param output_path: 输出CSV文件路径
    """
    try:
        # ========== 读取Excel文件 ==========
        df = pd.read_excel(input_path, engine='openpyxl')
        
        # ========== 数据验证 ==========
        required_cols = ['序号', '发帖强度', '互动率', '粉丝关注比', '高影响力用户']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"文件缺少必要列：{', '.join(missing_cols)}")

        # ========== 数据类型转换 ==========
        # 转换布尔列（处理Excel中的TRUE/FALSE可能被读取为字符串的情况）
        df['高影响力用户'] = df['高影响力用户'].astype(str).str.upper().map({'TRUE': 1, 'FALSE': 0}).astype(bool)

        # ========== 数值列归一化 ==========
        numeric_cols = ['发帖强度', '互动率', '粉丝关注比']
        df_normalized = df[numeric_cols].copy()
        
        # 创建归一化器函数
        def normalize_col(col_series):
            min_val = col_series.min()
            max_val = col_series.max()
            if max_val == min_val:
                return pd.Series([0.0]*len(col_series), name=col_series.name)
            return (col_series - min_val) / (max_val - min_val)
        
        # 应用归一化并保留4位小数
        df_normalized = df_normalized.apply(normalize_col).round(4)
        
        # ========== One-Hot编码 ==========
        influence_encoded = pd.get_dummies(
            df['高影响力用户'],
            prefix='高影响力用户',
            columns=['True', 'False'],
            dtype=int
        )
        
        # ========== 合并结果 ==========
        result = pd.concat([
            df[['序号']],
            df_normalized,
            influence_encoded
        ], axis=1)
        
        # ========== 保存结果 ==========
        result.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        # ========== 打印处理摘要 ==========
        print(f"成功处理 {len(df)} 条数据")
        print("各列归一化范围：")
        print(df_normalized.agg(['min', 'max']).round(4))
        print(f"结果文件已保存至：{output_path}")

    except Exception as e:
        print("="*40)
        print("处理失败，错误信息：")
        print(str(e))
        print("="*40)
        print("常见问题排查：")
        print("1. 确认Excel文件格式为.xlsx（2007+版本）")
        print("2. 检查Excel中是否包含隐藏字符或合并单元格")
        print("3. 验证数值列是否包含非数字内容")

# 使用示例
if __name__ == "__main__":
    process_excel_data(
        input_path="E:/comorbidity/feature_list/用户社交特征.xlsx",  # 修改为实际路径
        output_path="E:/comorbidity/feature_normalization/user.csv"  # 修改为实际路径
    )

成功处理 95163 条数据
各列归一化范围：
     发帖强度  互动率  粉丝关注比
min   0.0  0.0    0.0
max   1.0  1.0    1.0
结果文件已保存至：E:/comorbidity/feature_normalization/user.csv


In [10]:
import pandas as pd

def process_excel_data(input_path, output_path):
    """
    处理Excel社交媒体数据
    :param input_path: 输入Excel文件路径
    :param output_path: 输出CSV文件路径
    """
    try:
        # ========== 读取Excel文件 ==========
        df = pd.read_excel(input_path, engine='openpyxl')
        
        # ========== 数据验证 ==========
        required_cols = ['序号', '发帖强度', '互动强度', '粉丝关注比']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"文件缺少必要列：{', '.join(missing_cols)}")

        # ========== 数据类型转换 ==========
        # 转换布尔列（处理Excel中的TRUE/FALSE可能被读取为字符串的情况）
        # df['高影响力用户'] = df['高影响力用户'].astype(str).str.upper().map({'TRUE': 1, 'FALSE': 0}).astype(bool)

        # ========== 数值列归一化 ==========
        numeric_cols = ['发帖强度', '互动强度', '粉丝关注比']
        df_normalized = df[numeric_cols].copy()
        
        # 创建归一化器函数
        def normalize_col(col_series):
            min_val = col_series.min()
            max_val = col_series.max()
            if max_val == min_val:
                return pd.Series([0.0]*len(col_series), name=col_series.name)
            return (col_series - min_val) / (max_val - min_val)
        
        # 应用归一化并保留4位小数
        df_normalized = df_normalized.apply(normalize_col).round(4)
        
        # ========== One-Hot编码 ==========
        # influence_encoded = pd.get_dummies(
        #     df['高影响力用户'],
        #     prefix='高影响力用户',
        #     columns=['True', 'False'],
        #     dtype=int
        # )
        
        # ========== 合并结果 ==========
        result = pd.concat([
            df[['序号']],
            df_normalized
        ], axis=1)
        
        # ========== 保存结果 ==========
        result.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        # ========== 打印处理摘要 ==========
        print(f"成功处理 {len(df)} 条数据")
        print("各列归一化范围：")
        print(df_normalized.agg(['min', 'max']).round(4))
        print(f"结果文件已保存至：{output_path}")

    except Exception as e:
        print("="*40)
        print("处理失败，错误信息：")
        print(str(e))
        print("="*40)
        print("常见问题排查：")
        print("1. 确认Excel文件格式为.xlsx（2007+版本）")
        print("2. 检查Excel中是否包含隐藏字符或合并单元格")
        print("3. 验证数值列是否包含非数字内容")

# 使用示例
if __name__ == "__main__":
    process_excel_data(
        input_path="E:/comorbidity/feature_list/用户社交特征33.xlsx",  # 修改为实际路径
        output_path="E:/comorbidity/feature_normalization/user33.csv"  # 修改为实际路径
    )

成功处理 95163 条数据
各列归一化范围：
     发帖强度  互动强度  粉丝关注比
min   0.0   0.0    0.0
max   1.0   1.0    1.0
结果文件已保存至：E:/comorbidity/feature_normalization/user33.csv
